In [2]:
import random as rand
import pandas as pd
import re
import emoji

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [3]:
rand.seed(0)

In [4]:
training_df = pd.read_csv("C:\\Users\\Thomas\\Desktop\\training_data.csv")

c:\Users\Thomas\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
training_df[:10]

,text,reply_to_screen_name,is_quote,is_retweet,hashtags,country
0,Remember the #WuhanCoronaVirus? The pandemic w...,NaN,False,True,WuhanCoronaVirus KillerCuomo,us
1,My sources @WhiteHouse say 2 tactics will be u...,NaN,False,True,Trump,us
2,I'll venture a wild guess: If you were running...,NaN,False,True,COVID19,us
3,#Pakistan (#GreenStimulus = #Nature protection...,NaN,False,True,Pakistan GreenStimulus Nature Green,us
4,🇺🇸 Pandémie de #coronavirus: 30 pasteurs améri...,NaN,False,True,coronavirus COVID__19 COVIDー19,us
5,So I made a meme....#corona #virus #meme #coro...,NaN,False,True,corona virus meme coronavirusmeme toilet paper...,us
6,When I first saw the revised CDC figure—which ...,NaN,False,True,Coronavirus,us
7,"Um, no one’s really listening to you. #blackli...",NaN,True,False,blacklifematters COVID19 StopPoliceBrutality,us
8,ஆக்கப்பூர்வமான யோசனையை @CMOTamilNadu அரசுக்கு ...,NaN,False,True,corona,us
9,The Academy Might Be Postponing The Oscars\n\n...,NaN,False,False,TheAcademy AcademyAwards Oscars TheOscars AMPA...,us


In [6]:
#training_df['text_remove_newline'] = training_df['text'].apply(lambda x: x.replace("\n", ""))
#training_df['text_demojize'] = training_df['text_remove_newline'].apply(lambda x: emoji.demojize(x, delimiters=("", " ")))

In [7]:

from tqdm import tqdm
from bs4 import BeautifulSoup
import requests

session = requests.Session()
tqdm.pandas()

In [8]:
allowlist = [
  'p',
  'h1',
  'h2',
  'div',
]

#expanding URLs
def get_URL_data(tweet):
	urls = re.findall(r'(https?://\S+)', tweet)
	for url in urls:
		try:
			tweet = tweet.replace(url, session.head(url, allow_redirects=True, timeout=3).url)
			
			#html = requests.get(url).text
			#print(html)
			#soup = BeautifulSoup(html, "lxml")
			#text_elements = [t for t in soup.find_all(text=True) if t.parent.name in allowlist]
			#text += " ".join(text_elements)
		except:
			continue
	
	#text = text.replace("\n", "")
	#text = text.replace("\t", "")
	#text = " ".join(text.split())
	#data = [tweet, text]
	
	return tweet

In [9]:
get_URL_data(training_df['text'][21495])

'So remember Sturgis rally...?\nPositives are rolling in.\n#COVID19\nhttps://www.google.com/amp/s/www.cbsnews.com/amp/news/coronavirus-sturgis-motorcycle-rally-south-dakota-over-100-cases-8-states/'

In [10]:
data = []
file_cnt = 1
for i in tqdm(range(len(training_df))):
	data.append(get_URL_data(training_df['text'][i]))
	
	if (i % 5000 == 0 and i != 0):
		df = pd.DataFrame(data, columns = ['text_full_url'])
		df.to_csv('data/training_data2_' + str(file_cnt) + '.csv', index = False)
		data = []
		file_cnt += 1



  1%|          | 2088/240000 [18:32<17:30:11,  3.78it/s] 

In [ ]:
#translate emoji to words
training_df['text_full_url'] = training_df['text_full_url'].apply(lambda x: emoji.demojize(x, delimiters = (" ", " ")))

In [ ]:
y = training_df['country']

In [ ]:
tfidf = TfidfVectorizer()
X_vec = tfidf.fit_transform(training_df['text_full_url'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_vec,y,test_size=0.2, random_state = 0)

NameError: name 'X_vec' is not defined

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.447875


In [ ]:
from sklearn.svm import LinearSVC

svm = LinearSVC()
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.44997916666666665


In [ ]:
from sklearn.linear_model import LogisticRegression


lr = LogisticRegression(solver = 'saga')
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.445
